In [1]:
import pandas as pd
import itertools as it
import gurobipy as gp
from gurobipy import GRB

In [39]:
input1 = [[0.1, 0.5, 0.4],
          [0.1, 0.1, 0.2],
          [0.2, 0.1, 0.2],
          [0.6, 0.3, 0.2]]

input2 = [[0.0, 0.7, 0.9],
          [0.1, 0.1, 0.2],
          [0.2, 0.1, 0.2],
          [0.4, 0.2, 0.1]]

industry = ['COAL', 'STEEL', 'TRANS']
indu_man = ['COAL', 'STEEL', 'TRANS', 'MANPO']
years = list(range(1, 7))
year_later1 = dict(zip(list(range(1, 6)), [t+1 for t in years]))
year_later2 = dict(zip(list(range(1, 6)), [t+2 for t in years]))
extrayears = list(range(1,8))

production = pd.DataFrame(input1, columns=industry, index=indu_man)
extra_prod = pd.DataFrame(input2, columns=industry, index=indu_man)

init_stock = dict(zip(industry, [150, 80, 100]))
init_capac = dict(zip(industry, [300, 350, 280]))
exo_consum = dict(zip(industry, [60, 60, 30]))

### Static Model - Year 6 (and beyond)

Since in the final model we will have decision variables that concern year 6 and beyond, we need to solve these variables as a static economic model (as if it were only one year, a single period model). So, in the model for 5 years, we set the values of variables concerning year 6 and beyond as the value found in the solution of the static model.

In [3]:
model_static = gp.Model('Economic Planning Static')

# add vars
output_static = model_static.addVars(industry,
                       name='output_static')

# objective function
    # a constant
model_static.setObjective(0)

# add constraints
    # inputs in years beyond year 5
model_static.addConstrs((gp.quicksum(production[o][i]*output_static[o] for o in industry)
                   == output_static[i] - exo_consum[i] for i in industry),
                name='input_static')

model_static.update()
model_static.write('Economic Planning Static.lp')
model_static.optimize()


--------------------------------------------
--------------------------------------------

Using license file C:\Users\Nara\gurobi.lic
Academic license - for non-commercial use only - expires 2021-03-12
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 3 rows, 3 columns and 9 nonzeros
Model fingerprint: 0xf9d72bcd
Coefficient statistics:
  Matrix range     [1e-01, 9e-01]
  Objective range  [0e+00, 0e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+01, 6e+01]
Presolve removed 3 rows and 3 columns
Presolve time: 0.02s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.02 seconds
Optimal objective  0.000000000e+00


In [4]:
report_static = pd.DataFrame([], columns=['Production'], index=industry)

for i in industry:
    report_static['Production'][i] = output_static[i].x
print('In a static economy, the following should be produced')
report_static

In a static economy, the following should be produced


,Production
COAL,166.396761
STEEL,105.668016
TRANS,92.307692


### Multi-period Model

In [ ]:
    # no output in year 1
for i in industry:
    x[i,1].ub = 0
    
    # no extra capacity production before year 3
for i in industry:
    for t in years[:1]:
        y[i,t].ub = 0
        
model.addConstrs((x[i,t] <= init_capac[i] + gp.quicksum(y[i,l] for l in years if l<=t) 
          for i in industry for t in years),
         name='prod_cap')

In [52]:
model = gp.Model('Economic Planning Case 1')

# add vars
x = model.addVars(industry, years,       # output of industry i in year t
                  name='x')
s = model.addVars(industry, years,      # stock level of industry i in beginning of year t
                   name='s')
y = model.addVars(industry, years,      # increase in productive capacity in the end of year t
                   name='y')


            
    # output of year 6 and beyond can't be below the static model solution
for i in industry:
    x[i,6].lb = output_static[i].x
    y[i,6].lb = 0
    #y[i,1].ub = init_capac[i]

# objective function
    # Case 1: maximize productive capacity by the end of the 5 years
#model.setObjective(gp.quicksum(y[i,t] for i in industry for t in years if t<=5), GRB.MAXIMIZE)
model.setObjective(gp.quicksum(y[i,5] for i in industry), GRB.MAXIMIZE)

# add constraints

    # total input
for t in years[:5]:
    if t == 1:
        model.addConstrs((gp.quicksum(production[o][i]*x[o,t+1] for o in industry)
                         + gp.quicksum(extra_prod[o][i]*y[o,t+2] for o in industry)
                         + s[i,t] + exo_consum[i] - init_stock[i] - x[i,t]
                         == 0 for i in industry),
                        name='input_'+str(t))
    if t > 1 and t < 5:
        model.addConstrs((gp.quicksum(production[o][i]*x[o,t+1] for o in industry)
                         + gp.quicksum(extra_prod[o][i]*y[o,t+2] for o in industry)
                         + s[i,t] - s[i,t-1] + exo_consum[i] - x[i,t]
                         == 0 for i in industry),
                        name='input_'+str(t))
    if t == 5:
        model.addConstrs((gp.quicksum(production[o][i]*x[o,t+1] for o in industry)
                         - s[i,t-1] + s[i,t] + exo_consum[i] - x[i,t]
                         == 0 for i in industry),
                        name='input_'+str(t))
    
    # manpower limitation
for t in years[:5]:
    if t < 5:
        model.addConstr((gp.quicksum(production[i]['MANPO']*x[i,t+1] for i in industry)
                        + gp.quicksum(extra_prod[i]['MANPO']*y[i,t+2] for i in industry) <= 470),
                         name='manpower_'+str(t))
    if t == 5:
        model.addConstr((gp.quicksum(production[i]['MANPO']*x[i,t+1] for i in industry) <= 470),
                         name='manpower_'+str(t))
    
    # productive capacity
model.addConstrs((x[i,t] <= init_capac[i] + gp.quicksum(y[i,l] for l in years if l<=t) 
                  for i in industry for t in years),
                 name='prod_cap')

model.update()
model.write('Economic Planning Case 1.lp')
model.Params.DualReductions = 0
model.optimize()

Changed value of parameter DualReductions to 0
   Prev: 1  Min: 0  Max: 1  Default: 1
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 38 rows, 54 columns and 227 nonzeros
Model fingerprint: 0x22ba78b3
Coefficient statistics:
  Matrix range     [1e-01, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [9e+01, 2e+02]
  RHS range        [2e+01, 5e+02]
Presolve time: 0.01s
Presolved: 38 rows, 54 columns, 227 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.0000000e+30   3.200000e+30   3.000000e+00      0s
       9    4.7000000e+03   0.000000e+00   0.000000e+00      0s

Solved in 9 iterations and 0.02 seconds
Optimal objective  4.700000000e+03


In [53]:
index_tuples = list(it.product(['Capacity', 'Production', 'Stocks'], industry)) + [('Manpower','')]
m_index = pd.MultiIndex.from_tuples(index_tuples,
                                    names=['','Industry'])
report_1 = pd.DataFrame([], columns=['Year '+str(t) for t in years], index=m_index)

for i,t in x.keys():
    if t < 5:
        report_1['Year '+str(t)]['Production',i] = x[i,t].x
        report_1['Year '+str(t)]['Capacity',i] = y[i,t].x
        report_1['Year '+str(t)]['Stocks',i] = s[i,t].x
        report_1['Year '+str(t)]['Manpower',''] = sum(production[i]['MANPO']*x[i,t+1].x for i in industry) + sum(extra_prod[i]['MANPO']*y[i,t+2].x for i in industry)
    if t == 5:
        report_1['Year '+str(t)]['Production',i] = x[i,t].x
        report_1['Year '+str(t)]['Capacity',i] = y[i,t].x
        report_1['Year '+str(t)]['Stocks',i] = s[i,t].x
        report_1['Year '+str(t)]['Manpower',''] = sum(production[i]['MANPO']*x[i,t+1].x for i in industry)
    if t == 6:
        report_1['Year '+str(t)]['Production',i] = x[i,t].x
        report_1['Year '+str(t)]['Capacity',i] = y[i,t].x
        report_1['Year '+str(t)]['Stocks',i] = s[i,t].x
        #report_1['Year '+str(t)]['Manpower',''] = sum(production[i]['MANPO']*x[i,t+1].x for i in industry)

print('Optimal solution of {:.2f}'.format(model.ObjVal))
report_1

Optimal solution of 4700.00


Year 1       Year 2      Year 3      Year 4  Year 5  \
           Industry                                                             
Capacity   COAL      4186.396761          0.0         0.0         0.0     0.0   
           STEEL      855.668016          0.0         0.0         0.0     0.0   
           TRANS      772.307692          0.0         0.0         0.0  4700.0   
Production COAL      4486.396761          0.0         0.0         0.0     0.0   
           STEEL     1205.668016          0.0         0.0         0.0     0.0   
           TRANS     1052.307692          0.0         0.0         0.0     0.0   
Stocks     COAL      4576.396761  4516.396761  226.396761  166.396761     0.0   
           STEEL     1225.668016  1165.668016  165.668016  105.668016     0.0   
           TRANS     1122.307692  1092.307692  122.307692   92.307692     0.0   
Manpower                     0.0          0.0       470.0         0.0   150.0   

                         Year 6  
           Industry              
Capacity   COAL             0.0  
           STEEL            0.0  
           TRANS            0.0  
Production COAL      166.396761  
           STEEL     105.668016  
           TRANS      92.307692  
Stocks     COAL             0.0  
           STEEL            0.0  
           TRANS            0.0  
Manpower                    NaN

In [ ]:
    # Case 2: maximize output by the end of the 5 years
model.setObjective(gp.quicksum(x[i,t] for i in industry for t in [4, 5]), 
                   GRB.MAXIMIZE)

    # Case 3: maximize productive capacity by the end of the 5 years
model.setObjective((gp.quicksum(production[i]['MANPO']*x[i,t+1] for i in industry)
                + gp.quicksum(prod_capac[i]['MANPO']*y[i,t+2] for i in industry)
                  for t in years[:5]), GRB.MAXIMIZE)